## Задание №5

### Написать программу, которая для заданного n позволяет определить, сколько целых чисел между 1 и n не делятся на 11 или на 5.

```ASM
format ELF64

public _start
public exit

section '.data' writeable
    newline db 10
    place   db 1

section '.bss' writeable
    input_buffer dq 32

section '.text' executable

_start:
    mov rax, 0
    mov rdi, 0
    mov rsi, input_buffer
    mov rdx, 32
    syscall

    mov rsi, input_buffer
    call stoi
    mov rdi, rax

    call func

    call print

    mov rax, 1
    mov rdi, 1
    mov rsi, newline
    mov rdx, 1
    syscall

    mov rax, 60
    xor rdi, rdi
    syscall

func:
    mov rbx, rax

    mov rax, rdi
    xor rdx, rdx
    mov rcx, 5
    div rcx
    sub rbx, rax

    mov rax, rdi
    xor rdx, rdx
    mov rcx, 11
    div rcx
    sub rbx, rax

    mov rax, rdi
    xor rdx, rdx
    mov rcx, 55
    div rcx
    add rbx, rax

    mov rax, rbx
    ret

print:
    push rax
    push rbx
    push rcx
    push rdx
    push rsi
    push rdi

    xor rbx, rbx

    mov rcx, 10
    .loop:
        xor rdx, rdx
        div rcx

        push rdx
        inc rbx

        test rax, rax
        jnz .loop

    .print_loop:
        pop rax
        add rax, '0'
        mov [place], al

        push rbx

        mov rax, 1
        mov rdi, 1
        mov rsi, place
        mov rdx, 1
        syscall

        pop rbx
        dec rbx
        jnz .print_loop

        pop rdi
        pop rsi
        pop rdx
        pop rcx
        pop rbx
        pop rax
        ret

stoi:
    push rbx
    push rcx
    push rdx

    xor rax, rax
    xor rcx, rcx
    xor rdx, rdx

    .loop:
        movzx rcx, byte [rsi]
        test cl, cl
        jz .exit

        cmp cl, '0'
        jb .exit
        cmp cl, '9'
        ja .exit

        sub cl, '0'

        mov rbx, 10
        mul rbx

        add rax, rcx

        inc rsi
        jmp .loop

    .exit:
        pop rdx
        pop rcx
        pop rbx
        ret


```C
#include <stdio.h>

int main(int argc, char* argv[]) {
    int n;
    printf("Введите целое число: ");
    scanf("%d", &n);

    long long res = n - (n / 5) - (n / 11) + (n / 55);

    printf("%lld\n", res);
}


## Задание №10

### Разработать программу аутентификации пользователя: вводится пароль. Если пароль неверен, выдается сообщение «Невереный пароль» и пароль вводится снова. Повтор допускается до 5 раз. Если правильный пароль так и не будет введен, выдается сообщение "Неудача", если пароль подобран, то выдается сообщение "Вошли" 

```ASM
format ELF64
public _start

section '.data' writeable
    password      db "1234Artem", 0
    msg_success   db "Вошли", 10
    len_success   = $ - msg_success

    msg_incorrect db "Невереный пароль", 10
    len_incorrect = $ - msg_incorrect

    msg_failure   db "Неудача", 10
    len_failure   = $ - msg_failure

    attempts      db 5

section '.bss' writeable
    input_buffer dq 32

section '.text' executable
_start:
    mov bl, [attempts]

    attempt_loop:
        mov rax, 0
        mov rdi, 0
        mov rsi, input_buffer
        mov rdx, 32
        syscall

        mov byte [rsi + rax - 1], 0

        mov rdi, password
        mov rsi, input_buffer
        call strcmp

        cmp rax, 0
        je success_path

        jmp incorrect_path

    success_path:
        mov rax, 1
        mov rdi, 1
        mov rsi, msg_success
        mov rdx, len_success
        syscall
        jmp exit

    incorrect_path:
        mov rax, 1
        mov rdi, 1
        mov rsi, msg_incorrect
        mov rdx, len_incorrect
        syscall

        dec bl
        jnz attempt_loop

    failure_path:
        mov rax, 1
        mov rdi, 1
        mov rsi, msg_failure
        mov rdx, len_failure
        syscall
        jmp exit

    exit:
        mov rax, 60
        xor rdi, rdi
        syscall

strcmp:
    .loop:
        mov al, byte [rdi]
        mov dl, byte [rsi]
        cmp al, dl
        jne .not_equal

        cmp al, 0
        je .equal

        inc rdi
        inc rsi
        jmp .loop

    .not_equal:
        mov rax, 1
        ret

    .equal:
        mov rax, 0
        ret



```C
#include <stdio.h>
#include <string.h>

int main() {
    const char* password = "1234Artem";

    char input_buffer[32];

    int acess = 0;

    int count = 0;

    for (int i = 0; i < 5; ++i) {
        printf("Введите пароль:\n");
        if (fgets(input_buffer, sizeof(input_buffer), stdin) == NULL) {
            break;
        }

        if (strlen(input_buffer) > 0 && input_buffer[strlen(input_buffer) - 1] == '\n') {
            input_buffer[strlen(input_buffer) - 1] = '\0';
        }

        if (strcmp(password, input_buffer) == 0) {
            printf("Вошли");
            acess = 1;
            break;
        } else {
            acess = 0;
        }
    }

    if (acess == 0) {
        printf("Неудача");
    }
}


## Задание №12

### Реализовать функцию, которая для заданного числа n определяет, что все его цифры следуют в неубывающем порядке.

```ASM
format ELF64

public _start

section '.data' writeable
    newline db 10
    place   db 1

    msg_yes db "Да", 10
    len_yes = $ - msg_yes

    msg_no  db "Нет", 10
    len_no  = $ - msg_no
section '.bss' writeable
    input_buffer dq 32

section '.text' executable

_start:
    mov rax, 0
    mov rdi, 0
    mov rsi, input_buffer
    mov rdx, 32
    syscall

    mov byte [rsi + rax - 1], 0
    mov rsi, input_buffer
    call stoi

    mov rdi, rax
    call check_monotonous

    cmp rax, 1
    je .print_yes

    .print_no:
        mov rax, 1
        mov rdi, 1
        mov rsi, msg_no
        mov rdx, len_no
        syscall
        jmp .exit

    .print_yes:
        mov rbx, 1
        mov rdi, 1
        mov rsi, msg_yes
        mov rdx, len_yes
        syscall
        jmp .exit

    .exit:
        mov rax, 60
        xor rdi, rdi
        syscall

check_monotonous:
    mov rcx, 10

    .loop:
        cmp rdi, 0
        je .success

        mov rax, rdi

        xor rdx, rdx
        mov rbx, 10
        div rbx

        cmp rdx, rcx
        ja .failure

        mov rcx, rdx
        mov rdi, rax
        jmp .loop

    .failure:
        mov rax, 0
        ret

    .success:
        mov rax, 1
        ret
stoi:
    push rbx
    push rcx
    push rdx

    xor rax, rax
    xor rcx, rcx
    xor rdx, rdx

    .loop:
        movzx rcx, byte [rsi]
        test cl, cl
        jz .exit

        cmp cl, '0'
        jb .exit
        cmp cl, '9'
        ja .exit

        sub cl, '0'

        mov rbx, 10
        mul rbx

        add rax, rcx

        inc rsi
        jmp .loop

    .exit:
        pop rdx
        pop rcx
        pop rbx
        ret


```C
#include <stdio.h>

int main() {
    int n;

    scanf("%d", &n);

    int pred = 10;

    while (n != 0) {
        int ele = n % 10;
        n /= 10;

        if (pred < ele) {
            printf("Нет");
            return 0;
        }

        pred = ele;
    }

    printf("Да");
}
